In [ ]:
import os
import sys
sys.path.append(os.path.abspath(".."))

import torch
import numpy as np
import open3d as o3d
from lib.utils.chamfer3D.dist_chamfer_3D import chamfer_3DDist

In [16]:
def compute_fscore(dist1, dist2, threshold=0.001):
    """
    Calculates the F-score between two point clouds with the corresponding threshold value.
    :param dist1: Batch, N-Points
    :param dist2: Batch, N-Points
    :param th: float
    :return: fscore, precision, recall
    """
    # NB : In this depo, dist1 and dist2 are squared pointcloud euclidean
    # distances, so you should adapt the threshold accordingly.
    precision_1 = torch.mean((dist1 < threshold).float(), dim=1)
    precision_2 = torch.mean((dist2 < threshold).float(), dim=1)
    fscore = 2 * precision_1 * precision_2 / (precision_1 + precision_2)
    fscore[torch.isnan(fscore)] = 0
    return [fscore, precision_1, precision_2]

def compute_points_metrics(gt, pred):
    chamLoss = chamfer_3DDist()
    dist1, dist2, idx1, idx2 = chamLoss(
        torch.FloatTensor(gt[None, ...]).cuda(),
        torch.FloatTensor(pred[None, ...]).cuda(),
    )
    chamfer_dis = dist1.mean() + dist2.mean()
    chamfer_dis = chamfer_dis.cpu()
    f_score, precision, recall = compute_fscore(dist1, dist2, threshold=0.05)
    f_score = f_score.cpu()[0]
    return [chamfer_dis, f_score]


In [ ]:
id = 1947
pcd_gt_path = f"../output/default/test/scene_ks3/evals/30000/test/pointclouds/{id}_gt_pcd.ply"
pcd_render_path = pcd_gt_path.replace("_gt_pcd.ply", "_rendered_pcd.ply")

pcd_gt = o3d.io.read_point_cloud(pcd_gt_path)
pcd_render = o3d.io.read_point_cloud(pcd_render_path)

points_gt = torch.from_numpy(np.asarray(pcd_gt.points, dtype=np.float32))
points_render = torch.from_numpy(np.asarray(pcd_render.points, dtype=np.float32))

# points_gt = points_gt[torch.norm(points_gt, dim=1) > 1e-6]
# points_render = points_render[torch.norm(points_render, dim=1) > 1e-6]

print(points_gt.shape)
print(points_render.shape)

chamfer_dis, f_score = compute_points_metrics(points_gt, points_render)
print(chamfer_dis)
print(f_score)

In [ ]:
chamfer_dis, f_score = compute_points_metrics(points_gt, points_render)
print(chamfer_dis)
print(f_score)